In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
import seaborn as sns
import scipy.stats as scs
import matplotlib.pyplot as plt 


________________________________

### Загрузка .CSV из Tilda ###

In [ ]:
shh = pd.read_csv('Tilda_web_13_10.csv')
#shh = data[["email", "referer", "property_id"]]
shh = shh[shh["property_id"] == "webinar-data-science-13-10-22"] # вывожу только нужный вебинар

___________________


In [ ]:

#   db=shh[shh.duplicated()] #выведение дублей в отдельный столбец
res = shh.drop_duplicates(subset=['email'], keep='first') # удаление дубликатов из датафрейма

In [ ]:
test = ['тут был список тестовых имейлов'] 
# список тестовых мейлов вряд ли всех...

res = res[~res['email'].isin(test)] # удаление этого списка из рабочего датафрейма

In [ ]:
#shh1 = res[res["property_id"] == "webinar-data-science-13-10-22"] # вывожу только нужный вебинар
clear_mails = res[["email"]] # вывожу только нужные значения

a = clear_mails.email.str.lower() #смена регистра, для очистки всех дублей
clear_mails['email2'] = a
clear_mails = clear_mails.drop(['email'], axis = 1)
clear_mails = clear_mails.rename(columns={'email2': 'email'}) # переименовываю

In [ ]:
ref = res['referer'].str.split('?', expand=True) # раскидываю реферер по ютмкам через знак вопроса
ref.columns = ['referer', 'utm'] # даю название новым столбцам
ref1 = ref['utm'].str.split('&', expand=True) # разделяю теперь ютмки по & 
ref1.columns = ['source', 'medium' , 'campaign', 'content', 'term' , 'clid', 'wtf'] 

In [ ]:
for column in ref1.columns:
    ref1[column] = ref1[column].str.replace('utm_source=' ,' ')
    ref1[column] = ref1[column].str.replace('utm_medium=', ' ')  # цикл на удаление ключей меток
    ref1[column] = ref1[column].str.replace('utm_campaign=', ' ')
    ref1[column] = ref1[column].str.replace('#Inscrever','')

In [ ]:

#ref1['source'] = np.where (ref1['source'].str.startswith('fbclid='), 'facebook_w', ref1['source']) # решение Санька на удаление fbclid


In [ ]:
ref2 = ref1.fillna(value= '-') # заполняю None тирешками, чтобы считались тоже

In [ ]:
clear_utm = ref2[['source', 'medium', 'campaign']] # присваиваю новые названия

In [ ]:
Reg = pd.concat([clear_mails, clear_utm],axis=1) # конкатенирую две таблицы
Reg = Reg[['source', 'medium', 'campaign', 'email']] # меняю порядок столбцов
Reg = Reg.rename(columns={'email': 'email_tilda'}) # переименовываю
#Reg = Reg.sort_values(by = ['source'], ascending = False).reset_index(drop = True)
#Reg = Reg.set_index(['source','medium', 'campaign'])
#Reg.to_csv('test123_reg123.csv')

__________
### Загрузка .CSV из Webinars ###

In [ ]:
data5 = pd.read_csv('Atendees_13_10.csv')
Atendees = data5[['Email']]


b = Atendees.Email.str.lower() #смена регистра, для очистки всех дублей
Atendees['Email2'] = b
Atendees = Atendees.drop(['Email'], axis = 1)
Atendees = Atendees.rename(columns={'Email2': 'email_atendees'})
#Atendees.to_csv('test_aten.csv')

______


__________
### Загрузка .CSV из amo ###

In [ ]:
data6 = pd.read_csv('amo_webinar_13_10.csv')
amo = data6[['Work email']]
amo = amo.rename(columns={'Work email': 'email_amo'})

c = amo.email_amo.str.lower()
amo['email_amo2'] = c
amo = amo.drop(['email_amo'], axis = 1)
amo = amo.rename(columns={'email_amo2': 'email_amo'})


#amo.to_csv('amo_test.csv')

In [ ]:
testik  = Atendees.merge(amo, left_on=['email_atendees'], right_on=['email_amo'], how='left')

In [ ]:
final = pd.merge(Reg, testik, left_on=['email_tilda'], right_on=['email_atendees'], how='left')
    #]final.insert(5, 'two', '.')
    #final['two'] = np.where (final['email_atendees'].str.startswith('NaN'), 'NaN', final['two'])
    #final1 = pd.merge(final, amo, left_on=['email_atendees'], right_on=['email_amo'], how='left')
#final1 = final.groupby(['source', 'medium', 'campaign']).count()
    #final1 = final1.drop(['one'], axis=1)


In [ ]:
final.to_csv('Webinar_tst_WOW_REALLY!!!!!!.csv')